경찰청_전국 지구대 파출소 주소 현황_20231231  
https://www.data.go.kr/data/15077036/fileData.do?recommendDataYn=Y

In [4]:
import pandas as pd
import requests
import time

# 카카오 REST API 키 넣기
KAKAO_API_KEY = "dcebea19513a5f8977ab02b86e0d78c2"

headers = {
    "Authorization": f"KakaoAK {KAKAO_API_KEY}"
}

# 주소 → 좌표 변환 함수
def get_coords(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = {"query": address}
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()
        documents = result.get("documents")
        if documents:
            lat = float(documents[0]["y"])
            lng = float(documents[0]["x"])
            return lat, lng
    except Exception as e:
        print(f"주소 변환 오류: {address} -> {e}")
    return None, None

# CSV 파일 읽기 (주소 컬럼 이름 맞춰서 바꿔!)
df = pd.read_csv("data/경찰청_전국 지구대 파출소 주소 현황_20231231.csv", encoding='cp949')

# 위도/경도 컬럼 초기화
df["위도"] = None
df["경도"] = None

# 주소마다 좌표 변환 시도
for i, addr in enumerate(df["주소"]):
    lat, lng = get_coords(addr)
    df.at[i, "위도"] = lat
    df.at[i, "경도"] = lng
    print(f"{i+1}/{len(df)} - {addr} => 위도: {lat}, 경도: {lng}")
    time.sleep(0.2)  # API 호출 속도 제한 맞추기

# 결과 CSV 저장
df.to_csv("addresses_with_coords.csv", index=False, encoding="utf-8-sig")
print("좌표 변환 완료, CSV 저장됨")


1/2051 - 서울특별시 중구  을지로 234 => 위도: 37.5664734170529, 경도: 127.003943654119
2/2051 - 서울특별시 중구  퇴계로 375-1 => 위도: 37.5652684538307, 경도: 127.013376356837
3/2051 - 서울특별시 중구  동호로 5길 15, 약수지구대 => 위도: 37.5521442775352, 경도: 127.0123820902
4/2051 - 서울특별시 중구  다산로 248 (신당동, 신당파출소) => 위도: 37.5650442671787, 경도: 127.01638496365
5/2051 - 서울특별시 중구  동호로 261 => 위도: 37.5587633776035, 경도: 127.004802045236
6/2051 - 서울특별시 중구  퇴계로 178 => 위도: 37.5608715980974, 경도: 126.991580763073
7/2051 - 서울특별시 중구 충무로56-1 => 위도: 37.5661815238113, 경도: 126.992848341729
8/2051 - 서울특별시 종로구  종로17길 4  => 위도: 37.5705022157858, 경도: 126.988922852655
9/2051 - 서울특별시 종로구 삼일대로15길 19 1층(임시파출소) => 위도: 37.5686968013869, 경도: 126.986345479877
10/2051 - 서울특별시 종로구  세검정로 226  => 위도: 37.6000216905431, 경도: 126.958463372948
11/2051 - 서울특별시 종로구  평창문화로 46 => 위도: 37.6052486740828, 경도: 126.966627287785
12/2051 - 서울특별시 종로구  삼청로 66 => 위도: 37.5814010100419, 경도: 126.981140926337
13/2051 - 서울특별시 종로구  효자로 9 => 위도: 37.576770888086, 경도: 126.974006743819
14/2051 -

In [6]:
# CSV 파일 읽기
df = pd.read_csv("addresses_with_coords.csv")

# 위도 또는 경도 값이 비어있는 행만 필터링
missing_coords = df[df["위도"].isnull() | df["경도"].isnull()]

# 결과 출력
print(f"좌표 누락 행 수: {len(missing_coords)}")
print(missing_coords)

# 필요하면 따로 저장
missing_coords.to_csv("missing_coords.csv", index=False, encoding="utf-8-sig")


좌표 누락 행 수: 60
        연번    시도청    경찰서    관서명     구분                                주소  위도  \
161    162    서울청   서울강서     염창    지구대            서울특별시 강서구  공항대로59다길 80 NaN   
233    234    서울청   서울도봉    도봉1    파출소      서울특별시 도봉구  도봉로169길 43 도봉1파출소 NaN   
303    304    부산청   부산금정     범어    지구대  부산광역시 금정구  팔송로39길 56 (남산동) 범어지구대 NaN   
336    337    부산청   부산기장     철마    파출소            부산광역시 기장군 철마면 철마로 473  NaN   
412    413    인천청   인천중부    동인천    파출소                인천광역시 중구  참외전로 131 NaN   
416    417    인천청  인천미추홀     숭의    지구대                       용현동 488-109 NaN   
500    501    광주청   광주서부     염주    파출소             광주광역시 서구  염화로134번길 84 NaN   
533    534    대전청   대전대덕    신탄진    지구대              대전광역시 대덕구  대덕대로 1427 NaN   
571    572    울산청   울산울주     언양    파출소              울산광역시 울주군 언양읍 남천둑길 7 NaN   
584    585    세종청   세종북부     연동    파출소            세종특별자치시  부강면 청연로 606-1 NaN   
594      7  경기남부청   수원중부     행궁    파출소               경기도 수원시 찰달구 창룡대로 48 NaN   
629     42  경기남부청   성남수정  